In [ ]:
!pip install kaggle

In [2]:
#configurating the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d girinathrs211555/driving-license-datasets

In [ ]:
# extrcing the compossed Dataset
from zipfile import ZipFile
dataset = '/content/driving-license-datasets.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

# **Importing Dependencies**

In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
Driving_Licence_files = os.listdir('/content/Driving licence datasets/DRL')
print(Driving_Licence_files[0:5])

print(Driving_Licence_files[-5:])

In [ ]:
Not_Driving_Licence_files = os.listdir('/content/Driving licence datasets/NDRL')
print(Not_Driving_Licence_files[0:5])

print(Not_Driving_Licence_files[-5:])

In [ ]:
print('Number of with Driving_Licence_files:', len(Driving_Licence_files))
print('Number of without Not_Driving_Licence_files:', len(Not_Driving_Licence_files))

In [9]:
#create the labels

Driving_Licence_labels = [1]*5500

Not_Driving_Licence_labels = [0]*5500

In [10]:
# Checking wheather the labels are created as expected
print(Driving_Licence_labels [0:5])

print(Not_Driving_Licence_labels[0:5])

[1, 1, 1, 1, 1]
[0, 0, 0, 0, 0]


In [11]:
print(len(Driving_Licence_labels))
print(len(Not_Driving_Licence_labels))

5500
5500


In [ ]:
# I am Concanating  the  two class as labels

labels = Driving_Licence_labels + Not_Driving_Licence_labels

print(len(labels))
print(labels[0:5])
print(labels[-5:])

In [13]:
# Convert images to numpy arrays

Driving_Licence_path = '/content/Driving licence datasets/DRL/'

data = []
for img_file in Driving_Licence_files:

  image = Image.open(Driving_Licence_path + img_file)
  image = image.resize((128,128))
  image = image.convert("RGB")
  image = np.array(image)
  data.append(image)

Not_Driving_Licence_path = '/content/Driving licence datasets/NDRL/'

for img_file in Not_Driving_Licence_files:

  image = Image.open(Not_Driving_Licence_path + img_file)
  image = image.resize((128,128))
  image = image.convert("RGB")
  image = np.array(image)
  data.append(image)

In [14]:
# Coverting image list and the label list to numpy arrays

X = np.array(data)
Y = np.array(labels)

In [ ]:
type(X)

In [ ]:
type(Y)

In [ ]:
print(X.shape)
print(Y.shape)

In [18]:
print(Y)

[1 1 1 ... 0 0 0]


# **Train Test Split**

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

In [21]:
#Scaling the data

X_train_scaled = X_train/255

X_test_scaled = X_test/255

# **Building a Convolutional Neural Network(CNN)**

In [22]:
import tensorflow as tf
from tensorflow import keras

In [23]:
num_of_classes = 2

model = keras.Sequential()

model.add(keras.layers.Conv2D(35, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(75, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(100, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))


model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(num_of_classes, activation='sigmoid'))


In [24]:
# Compile the neural network

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [ ]:
#traing the neural network
history = model.fit(X_train_scaled, Y_train, validation_split=0.2, epochs=10)

In [ ]:
loss, accuracy = model.evaluate(X_test_scaled, Y_test)
print('Test Accuracy=', accuracy)

In [ ]:
h = history

#plot the loss value
plt.plot(h.history['loss'], label='train loss')
plt.plot(h.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

#plot the accuracy value
plt.plot(h.history['acc'], label='train accuracy')
plt.plot(h.history['val_acc'], label='validation accuracy')
plt.legend()
plt.show()

In [ ]:
input_image_path = input('Path of the image to be predicted: ')

input_image = cv2.imread(input_image_path)

cv2_imshow(input_image)

input_image_resized = cv2.resize(input_image, (128, 128))

input_image_scaled = input_image_resized/255

input_image_reshaped = np.reshape(input_image_scaled, [1,128,128,3])

input_prediction = model.predict(input_image_reshaped)

print(input_prediction)

input_pred_label = np.argmax(input_prediction)

print(input_pred_label)

if input_pred_label == 1:

  print('This is an Driving Licence')

else:

  print('This is Not Driving Licence')

In [ ]:
#model.save('Dynamite.h5')